In [3]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
from user_agents import user_agent_list
import sys

In [7]:
# Read the CSV file into a pandas DataFrame
df = pd.read_csv('bem_mechanical_PE_Practicing_Certificate_List.csv')
df

,Name_name,Google Link Redirect
0,ASSOC PROF. Ir. Dr. SAIDDI ALI FIRDAUS B MOHAM...,https://www.google.com/search?q=ASSOC+PROF.+Ir...
1,ASSOC. PROF. IR. DR. MOHD DANIAL BIN IBRAHIM,https://www.google.com/search?q=ASSOC.+PROF.+I...
2,BRIG. JEN. DATO' IR. HJ. NAJMUDDIN BIN HJ. JAA...,https://www.google.com/search?q=BRIG.+JEN.+DAT...
3,DATO' Ir. AHMAD BIN HASSAN,https://www.google.com/search?q=DATO'+Ir.+AHMA...
4,DATO' Ir. AHMAD MURAD BIN OMAR,https://www.google.com/search?q=DATO'+Ir.+AHMA...
...,...,...
1616,PROF. MADYA Ir. Dr. AHMAD RASDAN BIN ISMAIL,https://www.google.com/search?q=PROF.+MADYA+Ir...
1617,PROF. MADYA Ir. Dr. MOHAMED THARIQ BIN HAMEED ...,https://www.google.com/search?q=PROF.+MADYA+Ir...
1618,PROF. MADYA Ir. HAYATI BINTI ABDULLAH,https://www.google.com/search?q=PROF.+MADYA+Ir...
1619,PROF. MADYA Ir. ZAINAL ABIDIN BIN WAN CHIK,https://www.google.com/search?q=PROF.+MADYA+Ir...


In [10]:
df = df.drop(df.index[:1])
df = df.reset_index(drop=True)
df

,Name_name,Google Link Redirect
0,Ir. AHMAD KAMARUZZAMAN B MOHD BAKI,https://www.google.com/search?q=Ir.+AHMAD+KAMA...
1,Ir. AHMAD KHAIRIRI BIN ABDUL GHANI,https://www.google.com/search?q=Ir.+AHMAD+KHAI...
2,Ir. AHMAD KHAIRUDDIN BIN AHMAD KAMAL,https://www.google.com/search?q=Ir.+AHMAD+KHAI...
3,Ir. AHMAD LUTFI BIN SALIM,https://www.google.com/search?q=Ir.+AHMAD+LUTF...
4,Ir. AHMAD MASYHUR BIN JAHAYA,https://www.google.com/search?q=Ir.+AHMAD+MASY...
...,...,...
1523,PROF. MADYA Ir. Dr. AHMAD RASDAN BIN ISMAIL,https://www.google.com/search?q=PROF.+MADYA+Ir...
1524,PROF. MADYA Ir. Dr. MOHAMED THARIQ BIN HAMEED ...,https://www.google.com/search?q=PROF.+MADYA+Ir...
1525,PROF. MADYA Ir. HAYATI BINTI ABDULLAH,https://www.google.com/search?q=PROF.+MADYA+Ir...
1526,PROF. MADYA Ir. ZAINAL ABIDIN BIN WAN CHIK,https://www.google.com/search?q=PROF.+MADYA+Ir...


In [12]:
#test looping
df1 = df.head(1000)
df_result = []


count = 1
total_user_agents = len(user_agent_list)

user_agent = user_agent_list[count]
headers = {'User-Agent': user_agent}
# response = requests.get(df1['Google Link Redirect'][0],headers=headers)

#Iterating the Names
for index, url in df1.iterrows():           
    
    response = requests.get(url['Google Link Redirect'],headers=headers)
    print(response)
    print(f'No.{index} {url['Name_name']}')
    
    #to change user agent if 429
    while response.status_code == 429:
        if count != total_user_agents:
            
            print('Switching Agents...')
            user_agent = user_agent_list[count-1]
            headers = {'User-Agent': user_agent}
            response = requests.get(url['Google Link Redirect'],headers=headers)
            print(response)
            print(headers)
            print(count)
            print(total_user_agents)
            time.sleep(2)
            count=count+1
                    
        else:
            print('Out of user agents')
            sys.exit()
        
 
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, 'lxml')
        
        # Find and extract the first search result link
        
        search_results = soup.find_all('a')
        for link in search_results:
            href = link.get('href')
            if href and re.match(r'https://[a-zA-Z0-9-]+\.linkedin\.com', href):
                linkedin_profile = href.replace('/url?q=', '').split('&')[0]
                print(f'Linkedin Profile = {linkedin_profile}')
                df_result.append([{'Name':url['Name_name'],
                                    'LinkedIn Profile':linkedin_profile,
                                    'Response':response,
                                    'User Agent':user_agent}])
                
    else:
        print(response)
        print(headers)
        df_result.append([{'Name':url['Name_name'],
                                    'LinkedIn Profile':'Error',
                                    'Response':response,
                                    'User Agent':user_agent}])
        



    
    

<Response [429]>
No.0 Ir. AHMAD KAMARUZZAMAN B MOHD BAKI
Switching Agents...
<Response [429]>
{'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15'}
1
34
Switching Agents...
<Response [429]>
{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0'}
2
34
Switching Agents...
<Response [429]>
{'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'}
3
34
Switching Agents...


KeyboardInterrupt: 

In [79]:
df_result1 = pd.concat([pd.DataFrame(result) for result in df_result], ignore_index=True)
df_result1

,Name,LinkedIn Profile,Response,User Agent
0,ASSOC PROF. Ir. Dr. SAIDDI ALI FIRDAUS B MOHAM...,https://my.linkedin.com/in/saiddi-ali-firdaus-...,<Response [200]>,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:7...
1,ASSOC. PROF. IR. DR. MOHD DANIAL BIN IBRAHIM,https://my.linkedin.com/in/assoc-prof-ir-dr-m-...,<Response [200]>,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:7...
2,ASSOC. PROF. IR. DR. MOHD DANIAL BIN IBRAHIM,https://my.linkedin.com/in/assoc-prof-ir-dr-m-...,<Response [200]>,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:7...
3,ASSOC. PROF. IR. DR. MOHD DANIAL BIN IBRAHIM,https://www.linkedin.com/pub/dir/Danial/Ibrahim,<Response [200]>,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:7...
4,DATO' Ir. AHMAD MURAD BIN OMAR,https://my.linkedin.com/in/ahmad-murad-omar-95...,<Response [200]>,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:7...
...,...,...,...,...
197,Ir. AHMAD IZDIHAR BIN SUPAAT,https://my.linkedin.com/in/ahmad-izdihar-31a18a40,<Response [200]>,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:7...
198,Ir. AHMAD IZDIHAR BIN SUPAAT,https://my.linkedin.com/in/ahmad-izdihar-31a18a40,<Response [200]>,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:7...
199,Ir. AHMAD JAILANI BIN MANSOR,https://www.linkedin.com/posts/mohd-ariff-m-6b...,<Response [200]>,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:7...
200,Ir. AHMAD JAILANI BIN MANSOR,https://my.linkedin.com/in/ahmad-jailani-manso...,<Response [200]>,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:7...


In [82]:
df_result1['Name'].nunique()

75

In [80]:
# Define the path for the Excel file
excel_file_path = 'LinkedInProfileScraper.xlsx'

# Save the filtered DataFrame to an Excel file
df_result1.to_excel(excel_file_path, index=False)

print("Result data saved to", excel_file_path)

Result data saved to LinkedInProfileScraper.xlsx
